In [1]:
%load_ext autoreload
%autoreload 2

import sys

import collections
from parse_rest.connection import register
from parse_rest.datatypes import Object
import key
from manager import KeyStrokeManager

register(key.APP_ID, key.REST_API_KEY)

class DataChunk(Object):
    pass

In [51]:
allData = DataChunk.Query.all().filter(appVersion__exists=True, interTapDistances__exists=True).limit(1000)
acsaluData = allData.filter(userId='acsalu')
co273Data = allData.filter(userId='co273')
jeanData = allData.filter(userId='jean')

In [52]:
def label_stats(data):
    labels = map(lambda d: d.emotion, data)
    counter = collections.Counter(labels)
    return counter

In [54]:
print('allData: ', label_stats(allData), '  Total: ', len(allData))
print('acsaluData: ', label_stats(acsaluData), '  Total: ', len(acsaluData))
print('co273Data: ', label_stats(co273Data), '  Total: ', len(co273Data))
print('jeanData: ', label_stats(jeanData), '  Total: ', len(jeanData))

allData:  Counter({'Happy': 129, 'Calm': 90, 'Sad': 54, 'Angry': 5})   Total:  278
acsaluData:  Counter({'Calm': 10, 'Sad': 9, 'Happy': 9})   Total:  28
co273Data:  Counter({'Happy': 94, 'Calm': 68, 'Sad': 32, 'Angry': 5})   Total:  199
jeanData:  Counter({'Happy': 26, 'Sad': 13, 'Calm': 12})   Total:  51


In [75]:
whoseData = 'jean' # all | acsalu | co273 | jean
keyStrokeManager = KeyStrokeManager(jeanData)

# Logistic Regression

In [80]:
keyStrokeManager.logisticRegression()
print(keyStrokeManager.crossValidScore())

0.724945533769


# SVM

In [77]:
keyStrokeManager.svm()
print(keyStrokeManager.crossValidScore())

0.687908496732


# Naive Bayes

In [78]:
keyStrokeManager.naiveBayes()
print(keyStrokeManager.crossValidScore())

0.646241830065


# Random Forest

In [79]:
keyStrokeManager.randomForest()
print(keyStrokeManager.crossValidScore())

0.819716775599


# Export

In [81]:
m_path = 'pkl/'
keyStrokeManager.saveParams(m_path + whoseData + '_params.csv')
keyStrokeManager.saveModel(m_path + whoseData + '.pkl')